In [134]:
import pandas as pd
import numpy as np

In [135]:
df = pd.read_csv('data-cleaned.csv')
label=df["isNaturalCaused"]
df.drop(labels=["isNaturalCaused","fire_year", "true_cause", "general_cause_desc"], axis=1, inplace=True)


#Preprocessing:

categorical = [var for var in df.columns if df[var].dtype=='O']
numerical = [var for var in df.columns if var not in categorical]

print(categorical)
print(numerical)

for var in categorical:
  df[var] = df[var].str.replace(" ", "")
  print(df[var].value_counts())

df.info()

['fire_type', 'fire_position_on_slope', 'weather_conditions_over_fire', 'wind_direction']
['assessment_hectares', 'current_size', 'fire_spread_rate', 'temperature', 'relative_humidity', 'wind_speed']
Surface    16719
Ground      5022
Crown       1010
               1
Name: fire_type, dtype: int64
Flat         18384
Bottom        1388
Upper1/3      1070
Middle1/3      910
Lower1/3       760
Name: fire_position_on_slope, dtype: int64
Clear          10514
Cloudy          8121
CBWet           1502
CBDry           1414
Rainshowers      952
Name: weather_conditions_over_fire, dtype: int64
W      5103
CLM    3249
SW     2714
NW     2683
SE     2470
E      1820
S      1592
N      1553
NE     1314
Name: wind_direction, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25321 entries, 0 to 25320
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   assessment_hectares           25321

In [136]:
# clean up categorical null variable
for var in categorical:
  df[var]= df[var].fillna("N/A")
  print(df[var].value_counts())


# clean up numerical null variable using median
for var in numerical:
  df[var] = df[var].fillna(df[var].median())

df.isnull().sum()



Surface    16719
Ground      5022
N/A         2569
Crown       1010
               1
Name: fire_type, dtype: int64
Flat         18384
N/A           2809
Bottom        1388
Upper1/3      1070
Middle1/3      910
Lower1/3       760
Name: fire_position_on_slope, dtype: int64
Clear          10514
Cloudy          8121
N/A             2818
CBWet           1502
CBDry           1414
Rainshowers      952
Name: weather_conditions_over_fire, dtype: int64
W      5103
CLM    3249
N/A    2823
SW     2714
NW     2683
SE     2470
E      1820
S      1592
N      1553
NE     1314
Name: wind_direction, dtype: int64


assessment_hectares             0
current_size                    0
fire_spread_rate                0
fire_type                       0
fire_position_on_slope          0
weather_conditions_over_fire    0
temperature                     0
relative_humidity               0
wind_direction                  0
wind_speed                      0
dtype: int64

In [137]:
# Apply Labeling Encoding
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
for var in categorical:
  df[var] = le.fit_transform(df[var])

df.head()

,assessment_hectares,current_size,fire_spread_rate,fire_type,fire_position_on_slope,weather_conditions_over_fire,temperature,relative_humidity,wind_direction,wind_speed
0,0.01,0.10,0.0,4,1,2,18.0,10.0,8,2.0
1,0.20,0.20,0.0,4,2,2,12.0,22.0,8,10.0
2,0.50,0.50,0.0,4,0,2,12.0,22.0,8,10.0
3,0.01,0.01,0.0,4,1,2,12.0,22.0,8,10.0
4,0.10,0.10,0.1,4,1,2,6.0,37.0,8,2.0


In [138]:
# normalizing
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df)
scaled_features = scaler.transform(df)

In [139]:
df_scaled = pd.DataFrame(scaled_features,columns=df.columns)
df_scaled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25321 entries, 0 to 25320
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   assessment_hectares           25321 non-null  float64
 1   current_size                  25321 non-null  float64
 2   fire_spread_rate              25321 non-null  float64
 3   fire_type                     25321 non-null  float64
 4   fire_position_on_slope        25321 non-null  float64
 5   weather_conditions_over_fire  25321 non-null  float64
 6   temperature                   25321 non-null  float64
 7   relative_humidity             25321 non-null  float64
 8   wind_direction                25321 non-null  float64
 9   wind_speed                    25321 non-null  float64
dtypes: float64(10)
memory usage: 1.9 MB


In [140]:
# handling imbalanced data
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss

x = df_scaled
y = label

smk = SMOTETomek()
x_res,y_res = smk.fit_resample(x,y)

x_res.shape, y_res.shape

((31304, 10), (31304,))

In [141]:
from sklearn.model_selection import train_test_split

# train test split
x_train, x_test, y_train, y_test = train_test_split(x_res,y_res,test_size=0.20,shuffle=True)

print("X_train shape:",x_train.shape)
print("X_test shape:",x_test.shape)
print("y_train shape:",y_train.shape)
print("y_test shape:",y_test.shape)

X_train shape: (25043, 10)
X_test shape: (6261, 10)
y_train shape: (25043,)
y_test shape: (6261,)


In [142]:
# model result
result_dict_train = {}
result_dict_test = {}

In [143]:
# SVC model
from sklearn.ensemble import RandomForestClassifier
import time
random_forrest = RandomForestClassifier(n_jobs = -1)
start_time = time.time()
random_forrest.fit(x_train, y_train)
end_time = time.time()
elapsed_time = end_time - start_time

In [144]:
# Measure test acc and precision
from sklearn import metrics
from sklearn.metrics import classification_report

y_pred = random_forrest.predict(x_test)

cm = metrics.confusion_matrix(y_test, y_pred, labels=[0, 1])
cm_df = pd.DataFrame(cm, index=["Actual Human Caused", "Actual Natural Caused"], columns=["Predicted Human Caused", "Predicted Natural Caused"])
print(cm_df)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
classification_rep = metrics.classification_report(y_test, y_pred, digits=6, target_names=["Human Caused", "Natural Caused"])
print(classification_rep)
print(f"Fitting Time: {elapsed_time}s")

                       Predicted Human Caused  Predicted Natural Caused
Actual Human Caused                      2810                       362
Actual Natural Caused                     274                      2815
Accuracy: 0.898418782942022
Precision: 0.8860560276990872
                precision    recall  f1-score   support

  Human Caused   0.911154  0.885876  0.898338      3172
Natural Caused   0.886056  0.911298  0.898500      3089

      accuracy                       0.898419      6261
     macro avg   0.898605  0.898587  0.898419      6261
  weighted avg   0.898772  0.898419  0.898418      6261

Fitting Time: 0.04859304428100586s


In [145]:
#parameters tunning
# from sklearn.model_selection import GridSearchCV
# from sklearn import metrics
# from sklearnex import patch_sklearn, config_context
# patch_sklearn()

# with config_context(target_offload="gpu:0"):
#     grid = {
#     "n_estimators": [10, 50, 100, 250, 500, 750, 1000],
#     'criterion':["gini", "entropy", "log_loss"],
#     "min_samples_split":[2, 10, 100, 250, 1000],
#     'max_features' : ["sqrt","log2", None],
#     'min_impurity_decrease' : [0.0, 0.1, 0.2, 0.5, 10],
#     }   

#     random_forrest_cv  = RandomForestClassifier(n_jobs = -1)
#     random_forrest_cv = GridSearchCV(random_forrest_cv, grid, cv=5)
#     random_forrest_cv.fit(x_train,y_train)
#     print("Best Parameters:",random_forrest_cv.best_params_)
#     y_pred = random_forrest_cv.predict(x_test)

#     print("Train Score:",random_forrest_cv.best_score_)
#     print("Test Score:",random_forrest_cv.score(x_test,y_test))

#     metrics.confusion_matrix(y_test, y_pred)
#     print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
#     print("Precision:",metrics.precision_score(y_test, y_pred))
#     classification_rep = metrics.classification_report(y_test, y_pred, digits=6, target_names=["Human Caused", "Natural Caused"])
#     print(classification_rep)